In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.svm import SVR, SVC
from utils import read_neutouch_raw, infer, bin_neutouch_signal
import glob

In [10]:
#!unzip rod_tests_modified.zip
data_dir = 'rod_tests/rod_test1'
rods = ['rod20', 'rod30', 'rod50']

In [14]:
fnames = glob.glob(data_dir + '/' + rods[0] + '/*tact')
fnames.sort()

In [15]:
df_raw = read_neutouch_raw(fnames[0])

In [17]:
tapped_times = np.loadtxt(data_dir + '/' + rods[0] + '/times.txt')

In [29]:
time_past = 0.05
time_future = 0.25

def get_stats2(data_dir):
    
    interesting_taxels = [32, 22, 7, 17, 52, 42, 67, 77]
    
    for rod in rods:
        fnames = glob.glob(data_dir + '/' + rod + '/*tact')
        fnames.sort()
        tapped_times = np.loadtxt(data_dir + '/' + rod + '/times.txt')

        result = np.zeros((1, len(interesting_taxels) + 1))

        for i, fname in enumerate(fnames):
            tap_time = tapped_times[i]
            df_raw = read_neutouch_raw(fname)
            df_timespan = df_raw[(df_raw.t >= (tap_time - time_past)) & (df_raw.t < (tap_time + time_future))]
            spikes = [len(df_timespan[(df_timespan.taxel==interesting_taxel) & (df_timespan.isPos==0)]) for interesting_taxel in interesting_taxels]
            print(df_timespan.shape, spikes)
            result = np.vstack((result, np.zeros((1, len(interesting_taxels) + 1))))
            result[-1, :-1] = np.array(spikes)
            result[-1, -1] = i/5
            
        #print(result)


In [30]:
get_stats2(data_dir)

(272, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(4, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(177, 3) [0, 0, 5, 0, 0, 0, 0, 15]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(992, 3) [0, 0, 12, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]
(0, 3) [0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def get_stats(tool_length, trial):

    df_raw = read_neutouch_raw(f'/datasets/sensory_ext/data/neutouch/tool_neutouch_1k/trial{trial}_{tool_length}.tact')
    df_essentials = pd.read_csv(f'/datasets/sensory_ext/data/neutouch/tool_neutouch_1k/trial{trial}_{tool_length}_essentials.csv')
    df_essentials = df_essentials[df_essentials.t <= df_raw.t.iloc[-1]]
    tapped_times = df_essentials.t.values
    interesting_taxels = [32, 22, 7, 17, 52, 42, 67, 77]
    result = np.zeros((1, len(interesting_taxels) + 1))
    
    for i, tap_time in enumerate(tapped_times):
        df_timespan = df_raw[(df_raw.t >= (tap_time - time_past)) & (df_raw.t < (tap_time + time_future))]
        spikes = [len(df_timespan[(df_timespan.taxel==interesting_taxel) & (df_timespan.isPos==0)]) for interesting_taxel in interesting_taxels]
        if df_essentials.label_y[i] < 0.02 or (df_essentials.label_y[i] > 0.08 and df_essentials.label_y[i] < 0.12) or (df_essentials.label_y[i] > 0.18 and df_essentials.label_y[i] < 0.22) or (df_essentials.label_y[i] > 0.28 and df_essentials.label_y[i] < 0.32) or (df_essentials.label_y[i] > 0.38 and df_essentials.label_y[i] < 0.42) or df_essentials.label_y[i] > 0.48:
            continue
        result = np.vstack((result, np.zeros((1, len(interesting_taxels) + 1))))
        result[-1, :-1] = np.array(spikes)
        result[-1, -1] = int(df_essentials.label_y[i] * 100) // 10

    stats = np.zeros((2, len(interesting_taxels) + 1))
    stats[0, :] = np.mean(result[result[:, -1] == 0], 0)
    stats[1, :] = np.mean(result[result[:, -1] == 1], 0)
#     stats[2, :] = np.mean(result[result[:, -1] == 2], 0)
#     stats[3, :] = np.mean(result[result[:, -1] == 3], 0)
#     stats[4, :] = np.mean(result[result[:, -1] == 4], 0)

    return stats

output = np.zeros((20, 9))
for trial in range(1, 11):
    output[2*(trial-1):2*trial] = get_stats(20, trial)